We start by importing our dependancies and dataset

In [34]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
# from quilt.data.ResidentMario import missingno_data
# import missingno as msno

import seaborn as sns


# Machine learning
# import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# from catboost import CatBoostClassifier, Pool, cv

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize



# from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


from sklearn import metrics

# the command below means that the output of multiple commands in a cell will be output at once.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [57]:
test = pd.read_csv('/Users/anthonymiyoro/Desktop/rossman_store/data/test.csv')
train_store = pd.read_csv('/Users/anthonymiyoro/Desktop/rossman_store/data/train_store.csv')

In [58]:
train_store = train_store.fillna(0)

In [59]:
train_store.head(5)
# train_store.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,1,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,1,570.0,11.0,2007.0,1,13.0,2010.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,1,14130.0,12.0,2006.0,1,14.0,2011.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,3,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,1,29910.0,4.0,2015.0,0,0.0,0.0,0


In [60]:
train_store.loc[train_store['StoreType'] == 'a', 'StoreType'] = 1
train_store.loc[train_store['StoreType'] == 'b', 'StoreType'] = 2
train_store.loc[train_store['StoreType'] == 'c', 'StoreType'] = 3
train_store.loc[train_store['StoreType'] == 'd', 'StoreType'] = 4
train_store['StoreType'] = train_store['StoreType'].astype(int, copy=False)

In [39]:
# print('levels :', train_store['StoreType'].unique(), '; data type :', train_store['StoreType'].dtype)

When building our model, we cannot use a feature with the datetime data type as all features need to be numbers. We will then convert the Datetimes in the Date column to numbers in separate columns.

In [61]:
train_store['Date']=pd.to_datetime(train_store['Date'],dayfirst=False)

In [62]:
Date = train_store['Date']

date_df = pd.DataFrame({"year": Date.dt.year,
              "month": Date.dt.month,
              "day": Date.dt.day,
              #"hour": Date.dt.hour,
#               "dayofyear": Date.dt.dayofyear,
              "week": Date.dt.week,
                        "Date":Date.dt.date,
#               "weekofyear": Date.dt.weekofyear,
#               "dayofweek": Date.dt.dayofweek,
#               "weekday": Date.dt.weekday,
#               "quarter": Date.dt.quarter,
             })

In [63]:
date_df.head()

,Date,day,month,week,year
0,2015-07-31,31,7,31,2015
1,2015-07-31,31,7,31,2015
2,2015-07-31,31,7,31,2015
3,2015-07-31,31,7,31,2015
4,2015-07-31,31,7,31,2015


In [64]:
new_df = pd.merge(date_df, train_store, left_on='Date', right_on='Date')


In [65]:
new_df.head(5)

,Date,day,month,week,year,Store,DayOfWeek,Sales,Customers,Open,...,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


## We will begin by trying to select only the features that will explain the performance of our model:

## Checking features for Multicollinearity

We will use a collinearity plot to check for features that are heavily correlated to each other.

Multicollinearity is a problem when we have highly correlated explanatory features, as this makes it difficult for the model to try to differentitae between each feature's influence on the explanatory feature.
In the code below, instead of plotting the correlation matrix, we filter to only the top abs correlations that are not one. This will help us isolate the interactions that are most problematic.

The function below from https://stackoverflow.com/a/43104383 displays and drops the highest correlated features from our dataframe relative to the correlation value we give it.

In [42]:
def corr_df(x, corr_val):
    '''
    Obj: Drops features that are strongly correlated to other features.
          This lowers model complexity, and aids in generalizing the model.
    Inputs:
          df: features df (x)
          corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
    Output: df that only includes uncorrelated features
    '''

    # Creates Correlation Matrix and Instantiates
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterates through Correlation Matrix Table to find correlated columns
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = item.values
            if val >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(i)

    drops = sorted(set(drop_cols))[::-1]

    # Drops the correlated columns
    for i in drops:
        col = x.iloc[:, (i+1):(i+2)].columns.values
        df = x.drop(col, axis=1)

    return df

print("Top Absolute Correlations")
corr_df(train_store, 0.8)


Top Absolute Correlations
Promo2SinceYear | Promo2 | 1.0
PromoInterval | Promo2 | 0.82


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,1,1,570.0,11.0,2007.0,1,2010.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,1,1,14130.0,12.0,2006.0,1,2011.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,3,3,620.0,9.0,2009.0,0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,1,1,29910.0,4.0,2015.0,0,0.0,0
5,6,5,2015-07-31,5651,589,1,1,0,1,1,1,310.0,12.0,2013.0,0,0.0,0
6,7,5,2015-07-31,15344,1414,1,1,0,1,1,3,24000.0,4.0,2013.0,0,0.0,0
7,8,5,2015-07-31,8492,833,1,1,0,1,1,1,7520.0,10.0,2014.0,0,0.0,0
8,9,5,2015-07-31,8565,687,1,1,0,1,1,3,2030.0,8.0,2000.0,0,0.0,0
9,10,5,2015-07-31,7185,681,1,1,0,1,1,1,3160.0,9.0,2009.0,0,0.0,0


According to the function above, there is a high correlation between Promo2SinceYear | Promo2 and PromoInterval | Promo2. According to the feature documantation, Promo2 indicates a continuing promotion going on in a store while PromoInterval describes the consecutive intervals Promo2 is started. Promo2SinceYear indicates the time when the store started participating in the Promo2. With a correlation of 1.0, we will remove Promo2SinceYear.

In [43]:
del train_store['Promo2SinceYear']

In [44]:
train_store.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,1,1,570.0,11.0,2007.0,1,13.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,1,1,14130.0,12.0,2006.0,1,14.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,3,3,620.0,9.0,2009.0,0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,1,1,29910.0,4.0,2015.0,0,0.0,0


### We now split data into Train and Test. This allows us to measure the accuracy of our model when using both the testing and training data set.

In [45]:
print (train_store.columns)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'PromoInterval'],
      dtype='object')


In [46]:
train_store_copy = train_store
train_store_copy.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,1,1,570.0,11.0,2007.0,1,13.0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,1,1,14130.0,12.0,2006.0,1,14.0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,3,3,620.0,9.0,2009.0,0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,1,1,29910.0,4.0,2015.0,0,0.0,0


In [47]:
train, test = train_test_split(train_store_copy, test_size=0.2, random_state=42)
len(train)
len(test)

813767

203442

### We now separate our explanatory feature from our training features as is required by sci-kit learn. We will also separate our other features into train and test.

In [48]:
# create a training array = all the features but not the target 
# create a training test array = the results of the target variable for the training array
rf_trainY = train_store_copy['Sales']
rf_trainX = train_store_copy.drop('Sales', axis=1)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(rf_trainX, rf_trainY, test_size=0.2, random_state=42)

len(X_train)
len(X_test)
len(y_train)
len(y_test)

813767

203442

813767

203442

In [50]:
# Step 1: Initiating the bagging regressor algorithm
rf_regressor = RandomForestRegressor(n_estimators=100, oob_score=True)
# Step 2: Training the bagging regressor algorithm
rf_regressor.fit(X_train,y_train)
# Step 3: Calculating the score of the predictive power on the training and testing dataset.
print_scores(rf_regressor, X_train, y_train, X_test, y_test, oob_score=True)

TypeError: float() argument must be a string or a number, not 'Timestamp'